- DDPG는 연속적 행동 공간에서 결정론적 정책을 학습하는 알고리즘입니다.
- 여기서는 포트폴리오의 제약(각 자산 비중이 0 이상, 총합 1)을 만족하기 위해 Actor 네트워크의 출력을 softmax를 통해 정규화합니다.
- 탐험을 위해 학습 시에는 Actor의 출력에 Gaussian 노이즈를 추가합니다.
- 경험 재현(Replay Buffer)과 타깃 네트워크(target network)의 soft update를 사용합니다.

DDPG 에이전트 구성:

- Actor 네트워크: 상태를 입력받아 raw logits를 출력한 후 softmax를 적용하여, 각 자산의 투자 비중(0 이상, 합=1)을 생성합니다.
- Critic 네트워크: 상태와 행동을 입력받아 Q-value를 출력합니다.
- 타깃 네트워크와 경험 재현(Replay Buffer)을 사용하며, 학습 중에는 탐험을 위해 Gaussian 노이즈를 추가합니다.

학습 및 업데이트:

- Critic은 TD 목표와 현재 Q값 간의 MSE 손실로 업데이트합니다.
- Actor는 Critic의 Q값을 최대화하도록 업데이트합니다.
- 타깃 네트워크는 소프트 업데이트 방식(𝜏 τ)으로 주기적으로 갱신됩니다.

In [ ]:
# PoC 간단한 포트폴리오 최적화 via DDPG
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import yfinance as yf
import datetime
import random

import gym
from gym import spaces

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

##############################################
# 1. 데이터 다운로드 및 전처리
##############################################
# S&P500 상위 20종목 (예시 티커)
tickers = ["AAPL", "MSFT", "AMZN", "GOOGL", "META", "TSLA", "NVDA", "BRK-B", 
           "JNJ", "UNH", "V", "MA", "HD", "PG", "JPM", "BAC", "VZ", "DIS", "PFE", "XOM"]

# 10년 전부터 오늘까지의 데이터
end_date = datetime.datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.datetime.now() - datetime.timedelta(days=365*10)).strftime('%Y-%m-%d')

data = yf.download(tickers, start=start_date, end=end_date)["Close"]
data.dropna(inplace=True)

# 일별 수익률 계산
returns_df = data.pct_change().dropna()

##############################################
# 2. 벤치마크: 시가총액 비중 배분 계산
##############################################
market_caps = []
for ticker in tickers:
    try:
        info = yf.Ticker(ticker).info
        cap = info.get("marketCap", None)
        if cap is None:
            cap = 1e9
        market_caps.append(cap)
    except Exception as e:
        market_caps.append(1e9)

market_caps = np.array(market_caps)
benchmark_weights = market_caps / market_caps.sum()
print("Benchmark weights (시가총액 비중):")
for t, w in zip(tickers, benchmark_weights):
    print(f"{t}: {w:.4f}")

##############################################
# 3. 환경(Environment) 정의
##############################################
class HistoricalPortfolioEnv(gym.Env):
    """
    과거 일별 수익률 데이터를 순차적으로 제공하는 환경.
    - 상태(state): 해당일의 각 종목 수익률 (vector, shape: [n_assets])
    - 행동(action): 각 자산에 할당할 투자 비중 (연속적, 단, 모든 원소 ≥0, 합=1)
    - 보상(reward): 선택한 포트폴리오의 당일 수익률 (각 자산 수익률과 배분 비중의 내적)
    """
    def __init__(self, returns):
        super(HistoricalPortfolioEnv, self).__init__()
        self.returns = returns  # numpy array, shape=(T, n_assets)
        self.n_assets = returns.shape[1]
        self.current_step = 0
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(self.n_assets,), dtype=np.float32)
        # 행동은 연속적 벡터 (실제 action은 Actor의 softmax 출력을 사용)
        self.action_space = spaces.Box(low=0, high=1, shape=(self.n_assets,), dtype=np.float32)
    
    def reset(self):
        self.current_step = 0
        return self.returns[self.current_step]
    
    def step(self, action):
        # 혹시라도 action이 정규화되어 있지 않다면 보정
        weights = action / (np.sum(action) + 1e-8)
        current_return = self.returns[self.current_step]
        reward = np.dot(weights, current_return)
        self.current_step += 1
        done = self.current_step >= len(self.returns)
        if not done:
            next_state = self.returns[self.current_step]
        else:
            next_state = np.zeros(self.n_assets)
        return next_state, reward, done, {}

# numpy array로 변환
returns_np = returns_df.values

##############################################
# 4. Experience Replay Buffer 구현
##############################################
class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []
        self.position = 0
    
    def push(self, state, action, reward, next_state, done):
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = (state, action, reward, next_state, done)
        self.position = (self.position + 1) % self.capacity
    
    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        state, action, reward, next_state, done = map(np.stack, zip(*batch))
        return state, action, reward, next_state, done
    
    def __len__(self):
        return len(self.buffer)

##############################################
# 5. DDPG 모델 정의: Actor와 Critic 네트워크
##############################################
# Actor: 상태를 받아 raw logits를 출력하고 softmax를 통해 포트폴리오 비중 (합=1) 생성
class Actor(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.Softmax(dim=-1)
    
    # input-fc1-relu-fc2-relu-fc3-softmax
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        logits = self.fc3(x)
        action = self.softmax(logits)  # 보장: 모든 원소가 0 이상, 합=1
        return action

# Critic: 상태와 행동을 받아 Q-value 출력
class Critic(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(state_dim + action_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, 1)
    
    # input-fc1-relu-fc2-relu-fc3
    def forward(self, state, action):
        x = torch.cat([state, action], dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        q_value = self.fc3(x)
        return q_value

##############################################
# 6. 하이퍼파라미터 및 네트워크 초기화
##############################################
state_dim = returns_np.shape[1]    # n_assets
action_dim = returns_np.shape[1]     # 포트폴리오 비중 차원
hidden_dim = 64

# Actor와 Critic 네트워크
actor = Actor(state_dim, hidden_dim, action_dim)
critic = Critic(state_dim, action_dim, hidden_dim)

# 타깃 네트워크
target_actor = Actor(state_dim, hidden_dim, action_dim)
target_critic = Critic(state_dim, action_dim, hidden_dim)
target_actor.load_state_dict(actor.state_dict())
target_critic.load_state_dict(critic.state_dict())

# Optimizer
actor_optimizer = optim.Adam(actor.parameters(), lr=1e-4)
critic_optimizer = optim.Adam(critic.parameters(), lr=1e-3)

buffer_capacity = 10000
replay_buffer = ReplayBuffer(buffer_capacity)

# 기타 하이퍼파라미터
num_episodes = 50
batch_size = 64
gamma = 0.99
tau = 0.005  # 타깃 네트워크 soft update 비율

# 탐험을 위한 노이즈 (Gaussian noise)
def add_noise(action, noise_scale=0.1):
    noise = np.random.normal(0, noise_scale, size=action.shape)
    noisy_action = action + noise
    # softmax로 정규화해서 제약 조건(합=1, 각 원소 ≥ 0) 유지
    exp_action = np.exp(noisy_action)
    return exp_action / np.sum(exp_action)

##############################################
# 7. DDPG 학습 루프
##############################################
env = HistoricalPortfolioEnv(returns_np)

for episode in range(num_episodes):
    state = env.reset()
    episode_reward = 0
    done = False
    
    while not done:
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        actor.eval()
        with torch.no_grad():
            action = actor(state_tensor).cpu().data.numpy().flatten()
        actor.train()
        
        # 탐험을 위해 노이즈 추가 (학습 시)
        action = add_noise(action, noise_scale=0.1)
        
        next_state, reward, done, _ = env.step(action)
        episode_reward += reward
        
        replay_buffer.push(state, action, reward, next_state, done)
        state = next_state
        
        # 미니배치 업데이트
        if len(replay_buffer) >= batch_size:
            states_b, actions_b, rewards_b, next_states_b, dones_b = replay_buffer.sample(batch_size)
            
            states_b = torch.FloatTensor(states_b)
            actions_b = torch.FloatTensor(actions_b)
            rewards_b = torch.FloatTensor(rewards_b).unsqueeze(1)
            next_states_b = torch.FloatTensor(next_states_b)
            dones_b = torch.FloatTensor(dones_b).unsqueeze(1)
            
            # Critic 업데이트
            with torch.no_grad():
                next_actions = target_actor(next_states_b)
                target_q = target_critic(next_states_b, next_actions)
                y = rewards_b + gamma * target_q * (1 - dones_b)
            
            current_q = critic(states_b, actions_b)
            critic_loss = F.mse_loss(current_q, y)
            
            critic_optimizer.zero_grad()
            critic_loss.backward()
            critic_optimizer.step()
            
            # Actor 업데이트: maximize Q(s, actor(s))
            actor_loss = -critic(states_b, actor(states_b)).mean()
            actor_optimizer.zero_grad()
            actor_loss.backward()
            actor_optimizer.step()
            
            # 타깃 네트워크 soft update
            for target_param, param in zip(target_actor.parameters(), actor.parameters()):
                target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
            for target_param, param in zip(target_critic.parameters(), critic.parameters()):
                target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
    
    if episode % 5 == 0:
        print(f"Episode {episode:3d} | Episode Reward: {episode_reward:.4f}")

print("DDPG 학습 완료")

##############################################
# 8. 백테스팅: 학습된 Actor 네트워크로 전체 기간에 대해 포트폴리오 수익률 산출
##############################################
def backtest(actor_model, returns_np):
    actor_model.eval()
    env_bt = HistoricalPortfolioEnv(returns_np)
    state = env_bt.reset()
    daily_returns = []
    while True:
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        with torch.no_grad():
            action = actor_model(state_tensor).cpu().data.numpy().flatten()
        # 결정론적 행동: 노이즈 없이 actor의 출력 사용 (이미 softmax 적용됨)
        next_state, reward, done, _ = env_bt.step(action)
        daily_returns.append(reward)
        if done:
            break
        state = next_state
    return np.array(daily_returns)

ddpg_daily_returns = backtest(actor, returns_np)
ddpg_cumulative = np.cumprod(1 + ddpg_daily_returns)

##############################################
# 9. 벤치마크 백테스팅: BM (시가총액 비중 포트폴리오)
##############################################
# 벤치마크의 일별 수익률: 각 종목 수익률에 벤치마크 비중 곱합
benchmark_daily = returns_df.dot(benchmark_weights).values[-len(ddpg_daily_returns):]
benchmark_cumulative = np.cumprod(1 + benchmark_daily)

# 백테스트 기간 날짜 (returns_df의 index 사용)
dates = returns_df.index[-len(ddpg_daily_returns):]

##############################################
# 10. 결과 Plot: BM vs DDPG
##############################################
plt.figure(figsize=(12,6))
plt.plot(dates, benchmark_cumulative, label="Benchmark (BM)", linestyle="--")
plt.plot(dates, ddpg_cumulative, label="DDPG Agent")
plt.xlabel("날짜")
plt.ylabel("누적 수익률")
plt.title("Backtesting: BM vs DDPG")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 주석을 추가한 코드
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import yfinance as yf
import datetime
import random

import gym
from gym import spaces

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

##############################################
# 1. 데이터 다운로드 및 전처리
##############################################
# 포트폴리오에 사용할 S&P500 상위 20종목의 티커 리스트 (예시)
tickers = ["AAPL", "MSFT", "AMZN", "GOOGL", "META", "TSLA", "NVDA", "BRK-B", 
           "JNJ", "UNH", "V", "MA", "HD", "PG", "JPM", "BAC", "VZ", "DIS", "PFE", "XOM"]

# 시작일과 종료일을 설정: 오늘 날짜와 10년 전부터의 데이터를 다운로드
end_date = datetime.datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.datetime.now() - datetime.timedelta(days=365*10)).strftime('%Y-%m-%d')

data = yf.download(tickers, start=start_date, end=end_date)["Close"]
data.dropna(inplace=True)

# 각 날짜별로 각 종목의 수익률 계산 (전일 대비 변화율)
returns_df = data.pct_change().dropna()

##############################################
# 2. 벤치마크: 시가총액 비중 배분 계산
##############################################
# 각 종목의 시가총액 정보를 이용하여 벤치마크 포트폴리오의 비중을 계산
market_caps = []
for ticker in tickers:
    try:
        info = yf.Ticker(ticker).info
        cap = info.get("marketCap", None)
        if cap is None:
            cap = 1e9
        market_caps.append(cap)
    except Exception as e:
        # 오류 발생 시에도 임의의 값 할당 (데이터 누락 방지)
        market_caps.append(1e9)

market_caps = np.array(market_caps)
# 각 종목의 시가총액을 전체 시가총액 합으로 나누어 벤치마크 비중 계산
benchmark_weights = market_caps / market_caps.sum()
print("Benchmark weights (시가총액 비중):")
for t, w in zip(tickers, benchmark_weights):
    print(f"{t}: {w:.4f}")

##############################################
# 3. 환경(Environment) 정의
##############################################
class HistoricalPortfolioEnv(gym.Env):
    """
    과거 일별 수익률 데이터를 순차적으로 제공하는 환경.
    - 상태(state): 해당 일자의 각 종목의 수익률 (벡터, shape: [n_assets])
    - 행동(action): 각 자산에 할당할 투자 비중. 연속적인 값이며 모든 원소는 0 이상, 합이 1이 되도록 제약됨.
      (Actor 네트웤의 softmax 출력 결과를 사용)
    - 보상(reward): 해당 일자 포트폴리오의 수익률, 즉 각 종목의 수익률과 투자 비중의 내적(dot product)
    """
    def __init__(self, returns):
        super(HistoricalPortfolioEnv, self).__init__()
        self.returns = returns  # numpy array 형태로 일별 수익률 데이터, shape=(T, n_assets)
        self.n_assets = returns.shape[1]  # 종목의 수
        self.current_step = 0  # 에피소드 시작 시점 인덱스 초기화
        # 상태 공간: 각 종목의 수익률, 무한대 범위의 실수값 가능
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(self.n_assets,), dtype=np.float32)
        # 행동 공간: 각 종목에 할당할 비중, [0,1] 범위이며 실제로 softmax를 통해 정규화됨
        self.action_space = spaces.Box(low=0, high=1, shape=(self.n_assets,), dtype=np.float32)
    
    def reset(self):
        """에피소드 초기화: 시작 단계로 돌아가 현재 상태를 반환"""
        self.current_step = 0
        return self.returns[self.current_step]
    
    def step(self, action):
        """
        주어진 행동(투자 비중)에 따라 보상(당일 수익률)을 계산하고, 다음 상태로 전환.
        - 행동이 정규화되지 않은 경우, softmax와 유사하게 정규화해 합이 1이 되도록 보정.
        """
        # 행동 벡터의 합이 0이 되는 것을 방지하기 위해 작은 값 1e-8 추가
        weights = action / (np.sum(action) + 1e-8)
        # 현재 시점의 종목별 수익률
        current_return = self.returns[self.current_step]
        # 포트폴리오 수익률은 각 종목 수익률과 투자 비중의 내적
        reward = np.dot(weights, current_return)
        self.current_step += 1  # 다음 날짜로 이동
        # 에피소드 종료 여부 판단: 모든 데이터 소진 시 종료
        done = self.current_step >= len(self.returns)
        if not done:
            next_state = self.returns[self.current_step]
        else:
            next_state = np.zeros(self.n_assets)  # 에피소드 종료 시, 임의의 상태 반환
        return next_state, reward, done, {}

# Gym 환경에서 사용할 수 있도록 pandas DataFrame을 numpy array로 변환
returns_np = returns_df.values

##############################################
# 4. Experience Replay Buffer 구현
##############################################
class ReplayBuffer:
    """
    ReplayBuffer는 에이전트의 경험(상태, 행동, 보상, 다음 상태, 종료 여부)를 저장하여
    미니배치 학습 시 무작위로 샘플링할 수 있도록 지원한다.
    """
    def __init__(self, capacity):
        self.capacity = capacity  # 버퍼의 최대 저장 크기
        self.buffer = []         # 경험 저장 리스트
        self.position = 0        # 다음 저장할 인덱스
    
    def push(self, state, action, reward, next_state, done):
        """
        새로운 경험을 버퍼에 저장.
        버퍼가 가득 차면 순환 방식으로 기존 경험을 덮어쓴다.
        """
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = (state, action, reward, next_state, done)
        # 원형 버퍼를 위한 인덱스 업데이트
        self.position = (self.position + 1) % self.capacity
    
    def minibatch_sample(self, batch_size):
        """
        오리지널 코드는 함수명이 sample인데 기존 random.sample() 메서드랑 동명이라 헷갈려서 내가 바꿈
        저장된 경험 중 무작위로 미니배치 샘플링.
        반환 값은 각 항목(상태, 행동 등)을 numpy array로 묶어줌.
        """
        batch = random.sample(self.buffer, batch_size)
        state, action, reward, next_state, done = map(np.stack, zip(*batch))
        return state, action, reward, next_state, done
    
    def __len__(self):
        """현재 버퍼에 저장된 경험의 개수 반환"""
        return len(self.buffer)

##############################################
# 5. DDPG 모델 정의: Actor와 Critic 네트웤
##############################################
# Actor 네트웤: 상태를 입력받아 각 자산에 투자할 비중(확률 분포)를 출력
class Actor(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        # 출력 계층: 은닉층에서 포트폴리오 비중(로짓 값)으로 변환
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        # softmax를 사용하여 로짓을 확률 분포(합=1, 모든 원소 0 이상)로 변환
        self.softmax = nn.Softmax(dim=-1)
    
    # input-fc1-relu-fc2-relu-fc3-softmax
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # 출력 계층을 통해 raw logits 생성
        logits = self.fc3(x)
        # softmax 적용하여 투자 비중으로 정규화
        action = self.softmax(logits) # 보장: 모든 원소가 0 이상, 합=1
        return action

# Critic 네트웤: 주어진 상태와 행동에 대해 Q-value(예상 미래 보상)를 출력
class Critic(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(state_dim + action_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, 1)
    
    # input-fc1-relu-fc2-relu-fc3
    def forward(self, state, action):
        x = torch.cat([state, action], dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        q_value = self.fc3(x)
        return q_value

##############################################
# 6. 하이퍼파라미터 및 네트웤 초기화
##############################################
state_dim = returns_np.shape[1]    # 상태 차원: 자산의 개수
action_dim = returns_np.shape[1]     # 행동 차원: 포트폴리오 비중의 개수 (각 자산마다 하나씩)
hidden_dim = 64                      # 은닉층 차원

# Actor와 Critic 네트웤 인스턴스 생성
actor = Actor(state_dim, hidden_dim, action_dim)
critic = Critic(state_dim, action_dim, hidden_dim)

# 타깃 네트웤 생성: 안정적인 학습을 위해 원본 네트웤의 느린 업데이트 버전 사용
target_actor = Actor(state_dim, hidden_dim, action_dim)
target_critic = Critic(state_dim, action_dim, hidden_dim)
# 초기 타깃 네트웤 파라미터는 원본 네트웤(에서 copy하여)과 동일하게 설정
target_actor.load_state_dict(actor.state_dict())
target_critic.load_state_dict(critic.state_dict())

# Optimizer 설정: Actor와 Critic 네트웤 각각에 대해 Adam 옵티마이저 사용
actor_optimizer = optim.Adam(actor.parameters(), lr=1e-4)
critic_optimizer = optim.Adam(critic.parameters(), lr=1e-3)

# replay_buffer 초기화
buffer_capacity = 10000
replay_buffer = ReplayBuffer(buffer_capacity)

# 학습 관련 기타 하이퍼파라미터 설정
num_episodes = 50        # 전체 학습 에피소드 수
batch_size = 64          # 미니배치 샘플링 크기
gamma = 0.99
tau = 0.005              # 타깃 네트웤 soft update 비율 (원본 네트웤와 타깃 네트웤의 파라미터 업데이트 비율)
PRINT_INTERVAL = 5

# 탐험(exploration)을 위한 노이즈 함수: 행동에 Gaussian 노이즈를 추가한 후 softmax로 정규화
# Ornstein-Uhlenbeck Noise로 한다면 (Gaussian 노이즈 대비) 장단점은?
def add_noise(action, noise_scale=0.1):
    noise = np.random.normal(0, noise_scale, size=action.shape)
    noisy_action = action + noise
    # 노이즈가 추가된 행동을 다시 softmax 적용하여 제약 조건(모든 원소 0 이상, 합=1)을 만족
    exp_action = np.exp(noisy_action)
    return exp_action / np.sum(exp_action)

##############################################
# 7. DDPG 학습 루프
##############################################
env = HistoricalPortfolioEnv(returns_np)

for episode in range(num_episodes):
    state = env.reset() # 에피소드 시작 시 환경 초기화
    episode_reward = 0  # 해당 에피소드에서 누적된 보상
    done = False
    
    while not done:
        # 현재 상태를 tensor로 변환하여 Actor 네트웤에 입력
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        actor.eval()  # 평가 모드로 전환 (dropout, batchnorm 등이 있을 경우 영향을 줄 수 있음)
        with torch.no_grad():
            # Actor 네트웤가 출력한 투자 비중 (이미 softmax 적용됨)
            action = actor(state_tensor).cpu().data.numpy().flatten()
        actor.train()  # 다시 학습 모드로 전환
        
        # 학습 시, 탐험을 위해 노이즈 추가
        action = add_noise(action, noise_scale=0.1)
        
        # 환경에서 다음 상태, 보상, 종료 여부 획득
        next_state, reward, done, _ = env.step(action)
        episode_reward += reward
        
        # replay buffer 에 현재 경험 저장
        replay_buffer.push(state, action, reward, next_state, done)
        state = next_state
        
        # 충분한 경험이 쌓였으면, 미니배치로 네트웤 업데이트 수행
        if len(replay_buffer) >= batch_size:
            # 미니배치 샘플링
            states_b, actions_b, rewards_b, next_states_b, dones_b = replay_buffer.minibatch_sample(batch_size)
            
            # numpy array를 torch tensor로 변환
            states_b = torch.FloatTensor(states_b)
            actions_b = torch.FloatTensor(actions_b)
            rewards_b = torch.FloatTensor(rewards_b).unsqueeze(1)
            next_states_b = torch.FloatTensor(next_states_b)
            dones_b = torch.FloatTensor(dones_b).unsqueeze(1)
            
            # Critic 업데이트: 목표 Q-value 계산 및 MSE 손실 최소화
            with torch.no_grad():
                next_actions = target_actor(next_states_b) # 타깃 Actor를 사용해 다음 상태에서의 행동 예측
                target_q = target_critic(next_states_b, next_actions) # 타깃 Critic으로 다음 상태에서의 Q-value 계산
                # Bellman equation에 따른 목표 Q-value: 현재 보상 + 할인율 * 미래 Q-value * (종료 여부 반영)
                y = rewards_b + gamma * target_q * (1 - dones_b)
            
            # 현재 Critic 네트웤의 Q-value와 MSE-loss 계산
            current_q = critic(states_b, actions_b)
            critic_loss = F.mse_loss(current_q, y)
            
            # Critic 네트웤 파라미터 업데이트
            critic_optimizer.zero_grad()
            critic_loss.backward()
            critic_optimizer.step()
            
            # Actor 업데이트: 상태에서의 행동이 Critic 네트웤에서 높은 Q-value를 유도하도록 업데이트
            actor_loss = -critic(states_b, actor(states_b)).mean()
            actor_optimizer.zero_grad()
            actor_loss.backward()
            actor_optimizer.step()
            
            # 타깃 네트웤 soft update: 원본 네트웤 파라미터의 일부를 타깃 네트웤에 반영하여 천천히 업데이트
            """ 오리지널 코드인데 순서를 아래처럼 깔끔하게 변경했다
            for target_param, param in zip(target_actor.parameters(), actor.parameters()):
                target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
            for target_param, param in zip(target_critic.parameters(), critic.parameters()):
                target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
            """
            
            for param, target_param in zip(actor.parameters(), target_actor.parameters()):
                target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
            for param, target_param in zip(critic.parameters(), target_critic.parameters()):
                target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
    
    # 에피소드 진행 중 PRINT_INTERVAL 에피소드마다 결과 출력
    if episode % PRINT_INTERVAL == 0:
        print(f"Episode {episode:3d} | Episode Reward: {episode_reward:.4f}")

print("DDPG 학습 완료")

##############################################
# 8. 백테스팅: 학습된 Actor 네트웤으로 전체 기간에 대해 포트폴리오 수익률 산출
##############################################
def backtest(actor_model, returns_np):
    """
    주어진 학습된 Actor 네트웤을 사용하여 전체 백테스트 기간 동안의 일별 포트폴리오 수익률을 산출.
    - 결정론적 행동 사용: 탐험 노이즈 없이 Actor의 출력(softmax 적용된 투자 비중)을 그대로 사용.
    """
    actor_model.eval()  # 평가 모드로 전환
    env_bt = HistoricalPortfolioEnv(returns_np)  # 백테스트용 환경 초기화
    state = env_bt.reset()
    daily_returns = []
    while True:
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        with torch.no_grad():
            # 결정론적 행동 산출 (탐험 없이)
            action = actor_model(state_tensor).cpu().data.numpy().flatten()
        
        next_state, reward, done, _ = env_bt.step(action) # 환경에서 다음 상태, 보상, 종료 여부 획득
        daily_returns.append(reward)
        if done:
            break
        state = next_state
    return np.array(daily_returns)

# 백테스트: 학습된 DDPG Actor를 사용한 일별 수익률 산출
ddpg_daily_returns = backtest(actor, returns_np)
# 누적 수익률 계산: 매일의 수익률을 누적 곱하여 포트폴리오 성장 곡선 산출
ddpg_cumulative = np.cumprod(1 + ddpg_daily_returns)

##############################################
# 9. 벤치마크 백테스팅: BM (시가총액 비중 포트폴리오)
##############################################
# 벤치마크 포트폴리오의 일별 수익률 계산:
# 각 종목의 수익률에 시가총액 비중을 곱한 후 합산
benchmark_daily = returns_df.dot(benchmark_weights).values[-len(ddpg_daily_returns):]
# 누적 수익률 계산
benchmark_cumulative = np.cumprod(1 + benchmark_daily)

# 백테스트 기간 날짜 (returns_df의 index 중 마지막 부분을 사용)
dates = returns_df.index[-len(ddpg_daily_returns):]

##############################################
# 10. 결과 Plot: BM vs DDPG
##############################################
plt.figure(figsize=(12,6))
# 벤치마크 포트폴리오의 누적 수익률 그래프 (점선 스타일)
plt.plot(dates, benchmark_cumulative, label="Benchmark (BM)", linestyle="--")
# DDPG 에이전트 포트폴리오의 누적 수익률 그래프
plt.plot(dates, ddpg_cumulative, label="DDPG Agent")
plt.xlabel("Dates")
plt.ylabel("Cumulative returns")
plt.title("Backtesting: BM vs DDPG")
plt.legend()
plt.grid(True)
plt.show()

`Gaussian noise` 대신 `Ornstein-Uhlenbeck noise`를 사용하자.

이론적으로는:

`Gaussian 노이즈의 특징`
- 독립성: 각 시간 단계마다 생성되는 노이즈 값은 서로 독립적이다.
- 간단성: 구현과 이해가 상대적으로 쉽고, 추가 파라미터 없이 기본적인 정규분포에서 난수를 생성한다.
- 급격한 변화: 매 시간마다 독립적으로 노이즈가 추가되므로, 행동의 변화가 갑작스럽게 나타날 수 있다.

`OU 노이즈의 특징`
- 시간 상관성(`temporal correlations`): OU 노이즈는 이전 시간의 상태에 의존하여 노이즈가 생성되므로, 연속된 행동에 대해 부드럽고 점진적인 변화를 유도한다.
- 물리적 환경에 적합: 물리적 시스템이나 금융 시장과 같이 관성이 존재하는 환경에서는 갑작스러운 변화보다는 점진적인 변화가 더 자연스러울 수 있다.
- 추가 파라미터: `mu, theta, sigma`와 같이 몇 가지 추가 파라미터가 있어, 이를 적절하게 튜닝해야 한다.

장점 (OU 노이즈 사용 시)
- 부드러운 행동 변화: 시간 상관성이 있으므로, 액션이 갑자기 튀지 않고 연속적이다.
- 환경 적합성: 물리적 관성이 있거나, 연속적인 제어가 필요한 환경에서 효과적이다.

단점 (OU 노이즈 사용 시)
- 튜닝 복잡성: mu, theta, sigma 등의 파라미터 튜닝이 필요하며, 부적절한 값 선택 시 탐험이 과도하거나 부족할 수 있다.
- 과도한 상관성: 경우에 따라 너무 부드러운 탐험으로 인해 충분한 다양성이 확보되지 않을 수 있다.

### (Recall)  Ornstein-Uhlenbeck 프로세스

SDE:
$$ dX_{t} = \theta (\mu - X_{t}) dt + \sigma dW_{t} $$

여기서, 
- $X_t$ = 시간 𝑡에서의 노이즈 값
- $\mu$ = 장기 평균 (long-run mean)으로, 노이즈가 수렴하려는 값
- $\theta$ = 평균 회귀 속도 (mean reversion rate) $X_t$ 가 𝜇μ로 얼마나 빨리 회귀하는지를 결정한다. 값이 크면 노이즈가 빠르게 평균으로 돌아가, 과도한 편차를 억제한다.
- 𝜎σ = 변동성 (volatility) 또는 노이즈 강도, 난수 성분의 크기를 결정한다. 값이 크면 노이즈의 진폭이 커져서, 더 큰 무작위 변동이 발생한다.
- $dW_{t}$= 표준 Wiener 프로세스(브라운 운동)에서의 미소 변화로, 𝑁(0, 𝑑𝑡) N(0,dt)를 따르는 확률적 성분이다.

discrete time 에서의 근사:
실제로 구현할 때는 보통 시간 간격 𝑑𝑡=1로 두고 아래와 같이 근사한다:
$$ X_{t+1} = X_{t} + \theta ( \mu - X_{t}) + \sigma \epsilon $$
여기서 𝜖ϵ는 𝑁(0,1)를 따르는 정규 분포 난수다.

In [ ]:
# Gaussian noise 대신에 Ornstein-Uhlenbeck noise로 변경한 코드
# 주석을 추가한 코드
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import yfinance as yf
import datetime
import random

import gym
from gym import spaces

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

##############################################
# 1. 데이터 다운로드 및 전처리
##############################################
# 포트폴리오에 사용할 S&P500 상위 20종목의 티커 리스트 (예시)
tickers = ["AAPL", "MSFT", "AMZN", "GOOGL", "META", "TSLA", "NVDA", "BRK-B", 
           "JNJ", "UNH", "V", "MA", "HD", "PG", "JPM", "BAC", "VZ", "DIS", "PFE", "XOM"]

# 시작일과 종료일을 설정: 오늘 날짜와 10년 전부터의 데이터를 다운로드
end_date = datetime.datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.datetime.now() - datetime.timedelta(days=365*10)).strftime('%Y-%m-%d')

data = yf.download(tickers, start=start_date, end=end_date)["Close"]
data.dropna(inplace=True)

# 각 날짜별로 각 종목의 수익률 계산 (전일 대비 변화율)
returns_df = data.pct_change().dropna()

##############################################
# 2. 벤치마크: 시가총액 비중 배분 계산
##############################################
# 각 종목의 시가총액 정보를 이용하여 벤치마크 포트폴리오의 비중을 계산
market_caps = []
for ticker in tickers:
    try:
        info = yf.Ticker(ticker).info
        cap = info.get("marketCap", None)
        if cap is None:
            cap = 1e9
        market_caps.append(cap)
    except Exception as e:
        # 오류 발생 시에도 임의의 값 할당 (데이터 누락 방지)
        market_caps.append(1e9)

market_caps = np.array(market_caps)
# 각 종목의 시가총액을 전체 시가총액 합으로 나누어 벤치마크 비중 계산
benchmark_weights = market_caps / market_caps.sum()
print("Benchmark weights (시가총액 비중):")
for t, w in zip(tickers, benchmark_weights):
    print(f"{t}: {w:.4f}")

##############################################
# 3. 환경(Environment) 정의
##############################################
class HistoricalPortfolioEnv(gym.Env):
    """
    과거 일별 수익률 데이터를 순차적으로 제공하는 환경.
    - 상태(state): 해당 일자의 각 종목의 수익률 (벡터, shape: [n_assets])
    - 행동(action): 각 자산에 할당할 투자 비중. 연속적인 값이며 모든 원소는 0 이상, 합이 1이 되도록 제약됨.
      (Actor 네트웤의 softmax 출력 결과를 사용)
    - 보상(reward): 해당 일자 포트폴리오의 수익률, 즉 각 종목의 수익률과 투자 비중의 내적(dot product)
    """
    def __init__(self, returns):
        super(HistoricalPortfolioEnv, self).__init__()
        self.returns = returns  # numpy array 형태로 일별 수익률 데이터, shape=(T, n_assets)
        self.n_assets = returns.shape[1]  # 종목의 수
        self.current_step = 0  # 에피소드 시작 시점 인덱스 초기화
        # 상태 공간: 각 종목의 수익률, 무한대 범위의 실수값 가능
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(self.n_assets,), dtype=np.float32)
        # 행동 공간: 각 종목에 할당할 비중, [0,1] 범위이며 실제로 softmax를 통해 정규화됨
        self.action_space = spaces.Box(low=0, high=1, shape=(self.n_assets,), dtype=np.float32)
    
    def reset(self):
        """에피소드 초기화: 시작 단계로 돌아가 현재 상태를 반환"""
        self.current_step = 0
        return self.returns[self.current_step]
    
    def step(self, action):
        """
        주어진 행동(투자 비중)에 따라 보상(당일 수익률)을 계산하고, 다음 상태로 전환.
        - 행동이 정규화되지 않은 경우, softmax와 유사하게 정규화해 합이 1이 되도록 보정.
        """
        # 행동 벡터의 합이 0이 되는 것을 방지하기 위해 작은 값 1e-8 추가
        weights = action / (np.sum(action) + 1e-8)
        # 현재 시점의 종목별 수익률
        current_return = self.returns[self.current_step]
        # 포트폴리오 수익률은 각 종목 수익률과 투자 비중의 내적
        reward = np.dot(weights, current_return)
        self.current_step += 1  # 다음 날짜로 이동
        # 에피소드 종료 여부 판단: 모든 데이터 소진 시 종료
        done = self.current_step >= len(self.returns)
        if not done:
            next_state = self.returns[self.current_step]
        else:
            next_state = np.zeros(self.n_assets)  # 에피소드 종료 시, 임의의 상태 반환
        return next_state, reward, done, {}

# Gym 환경에서 사용할 수 있도록 pandas DataFrame을 numpy array로 변환
returns_np = returns_df.values

##############################################
# 4. Experience Replay Buffer 구현
##############################################
class ReplayBuffer:
    """
    ReplayBuffer는 에이전트의 경험(상태, 행동, 보상, 다음 상태, 종료 여부)를 저장하여
    미니배치 학습 시 무작위로 샘플링할 수 있도록 지원한다.
    """
    def __init__(self, capacity):
        self.capacity = capacity  # 버퍼의 최대 저장 크기
        self.buffer = []         # 경험 저장 리스트
        self.position = 0        # 다음 저장할 인덱스
    
    def push(self, state, action, reward, next_state, done):
        """
        새로운 경험을 버퍼에 저장.
        버퍼가 가득 차면 순환 방식으로 기존 경험을 덮어쓴다.
        """
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = (state, action, reward, next_state, done)
        # 원형 버퍼를 위한 인덱스 업데이트
        self.position = (self.position + 1) % self.capacity
    
    def minibatch_sample(self, batch_size):
        """
        오리지널 코드는 함수명이 sample인데 기존 random.sample() 메서드랑 동명이라 헷갈려서 내가 바꿈
        저장된 경험 중 무작위로 미니배치 샘플링.
        반환 값은 각 항목(상태, 행동 등)을 numpy array로 묶어줌.
        """
        batch = random.sample(self.buffer, batch_size)
        state, action, reward, next_state, done = map(np.stack, zip(*batch))
        return state, action, reward, next_state, done
    
    def __len__(self):
        """현재 버퍼에 저장된 경험의 개수 반환"""
        return len(self.buffer)

##############################################
# 5. DDPG 모델 정의: Actor와 Critic 네트웤
##############################################
# Actor 네트웤: 상태를 입력받아 각 자산에 투자할 비중(확률 분포)를 출력
class Actor(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        # 출력 계층: 은닉층에서 포트폴리오 비중(로짓 값)으로 변환
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        # softmax를 사용하여 로짓을 확률 분포(합=1, 모든 원소 0 이상)로 변환
        self.softmax = nn.Softmax(dim=-1)
    
    # input-fc1-relu-fc2-relu-fc3-softmax
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # 출력 계층을 통해 raw logits 생성
        logits = self.fc3(x)
        # softmax 적용하여 투자 비중으로 정규화
        action = self.softmax(logits) # 보장: 모든 원소가 0 이상, 합=1
        return action

# Critic 네트웤: 주어진 상태와 행동에 대해 Q-value(예상 미래 보상)를 출력
class Critic(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(state_dim + action_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, 1)
    
    # input-fc1-relu-fc2-relu-fc3
    def forward(self, state, action):
        x = torch.cat([state, action], dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        q_value = self.fc3(x)
        return q_value

##############################################
# 6. 하이퍼파라미터 및 네트웤 초기화
##############################################
state_dim = returns_np.shape[1]    # 상태 차원: 자산의 개수
action_dim = returns_np.shape[1]     # 행동 차원: 포트폴리오 비중의 개수 (각 자산마다 하나씩)
hidden_dim = 64                      # 은닉층 차원

# Actor와 Critic 네트웤 인스턴스 생성
actor = Actor(state_dim, hidden_dim, action_dim)
critic = Critic(state_dim, action_dim, hidden_dim)

# 타깃 네트웤 생성: 안정적인 학습을 위해 원본 네트웤의 느린 업데이트 버전 사용
target_actor = Actor(state_dim, hidden_dim, action_dim)
target_critic = Critic(state_dim, action_dim, hidden_dim)
# 초기 타깃 네트웤 파라미터는 원본 네트웤(에서 copy하여)과 동일하게 설정
target_actor.load_state_dict(actor.state_dict())
target_critic.load_state_dict(critic.state_dict())

# Optimizer 설정: Actor와 Critic 네트웤 각각에 대해 Adam 옵티마이저 사용
actor_optimizer = optim.Adam(actor.parameters(), lr=1e-4)
critic_optimizer = optim.Adam(critic.parameters(), lr=1e-3)

# replay_buffer 초기화
buffer_capacity = 10000
replay_buffer = ReplayBuffer(buffer_capacity)

# 학습 관련 기타 하이퍼파라미터 설정
num_episodes = 50        # 전체 학습 에피소드 수
batch_size = 64          # 미니배치 샘플링 크기
gamma = 0.99
tau = 0.005              # 타깃 네트웤 soft update 비율 (원본 네트웤와 타깃 네트웤의 파라미터 업데이트 비율)
PRINT_INTERVAL = 5

# 탐험(exploration)을 위한 노이즈 함수: 행동에 Gaussian 노이즈를 추가한 후 softmax로 정규화
# 기존의 add_noise 함수 대신 이름을 add_Gaussian_noise로 변경
def add_Gaussian_noise(action, noise_scale=0.1):
    noise = np.random.normal(0, noise_scale, size=action.shape)
    noisy_action = action + noise
    # 노이즈가 추가된 행동을 다시 softmax 적용하여 제약 조건(모든 원소 0 이상, 합=1)을 만족
    exp_action = np.exp(noisy_action)
    return exp_action / np.sum(exp_action)

# Ornstein-Uhlenbeck noise를 위한 클래스 정의 (내부 상태 유지)
# mu, theta, sigma 는 각각 노이즈의 장기 평균, 평균 회귀 속도(rate), 변동성(volatility) 또는 노이즈 강도인데 어떤 값으로 설정하는게 좋을지?
# 파라미터 튜닝 필요
class OUNoise:
    def __init__(self, action_dim, mu=0.0, theta=0.15, sigma=0.2):
        self.action_dim = action_dim
        self.mu = mu
        self.theta = theta
        self.sigma = sigma
        self.state = np.ones(self.action_dim) * self.mu
        
    def reset(self):
        self.state = np.ones(self.action_dim) * self.mu
        
    def __call__(self):
        dx = self.theta * (self.mu - self.state) + self.sigma * np.random.randn(self.action_dim)
        self.state = self.state + dx
        return self.state

# Ornstein-Uhlenbeck 노이즈를 추가하는 함수
def add_ornstein_uhlenbeck_noise(action, ou_noise):
    noise = ou_noise()  # OU 노이즈 프로세스를 통해 노이즈 획득
    noisy_action = action + noise
    exp_action = np.exp(noisy_action)
    return exp_action / np.sum(exp_action)

##############################################
# 7. DDPG 학습 루프
##############################################
env = HistoricalPortfolioEnv(returns_np)

# DDPG 학습 루프 시작 전, OU 노이즈 인스턴스 생성 (행동 차원에 맞춤)
ou_noise = OUNoise(action_dim, mu=0.0, theta=0.15, sigma=0.2)

for episode in range(num_episodes):
    state = env.reset() # 에피소드 시작 시 환경 초기화
    episode_reward = 0  # 해당 에피소드에서 누적된 보상
    done = False
    ou_noise.reset()    # 에피소드 시작 시 OU 노이즈 상태 초기화
    
    while not done:
        # 현재 상태를 tensor로 변환하여 Actor 네트웤에 입력
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        actor.eval()  # 평가 모드로 전환 (dropout, batchnorm 등이 있을 경우 영향을 줄 수 있음)
        with torch.no_grad():
            # Actor 네트웤가 출력한 투자 비중 (이미 softmax 적용됨)
            action = actor(state_tensor).cpu().data.numpy().flatten()
        actor.train()  # 다시 학습 모드로 전환
        
        """
        # 학습 시, 탐험을 위해 Gaussian 노이즈 추가
        action = add_Gaussian_noise(action, noise_scale=0.1)
        """
        # 기존 Gaussian 노이즈 대신 Ornstein-Uhlenbeck 노이즈 적용
        action = add_ornstein_uhlenbeck_noise(action, ou_noise)
        
        # 환경에서 다음 상태, 보상, 종료 여부 획득
        next_state, reward, done, _ = env.step(action)
        episode_reward += reward
        
        # replay buffer 에 현재 경험 저장
        replay_buffer.push(state, action, reward, next_state, done)
        state = next_state
        
        # 충분한 경험이 쌓였으면, 미니배치로 네트웤 업데이트 수행
        if len(replay_buffer) >= batch_size:
            # 미니배치 샘플링
            states_b, actions_b, rewards_b, next_states_b, dones_b = replay_buffer.minibatch_sample(batch_size)
            
            # numpy array를 torch tensor로 변환
            states_b = torch.FloatTensor(states_b)
            actions_b = torch.FloatTensor(actions_b)
            rewards_b = torch.FloatTensor(rewards_b).unsqueeze(1)
            next_states_b = torch.FloatTensor(next_states_b)
            dones_b = torch.FloatTensor(dones_b).unsqueeze(1)
            
            # Critic 업데이트: 목표 Q-value 계산 및 MSE 손실 최소화
            with torch.no_grad():
                next_actions = target_actor(next_states_b) # 타깃 Actor를 사용해 다음 상태에서의 행동 예측
                target_q = target_critic(next_states_b, next_actions) # 타깃 Critic으로 다음 상태에서의 Q-value 계산
                # Bellman equation에 따른 목표 Q-value: 현재 보상 + 할인율 * 미래 Q-value * (종료 여부 반영)
                y = rewards_b + gamma * target_q * (1 - dones_b)
            
            # 현재 Critic 네트웤의 Q-value와 MSE-loss 계산
            current_q = critic(states_b, actions_b)
            critic_loss = F.mse_loss(current_q, y)
            
            # Critic 네트웤 파라미터 업데이트
            critic_optimizer.zero_grad()
            critic_loss.backward()
            critic_optimizer.step()
            
            # Actor 업데이트: 상태에서의 행동이 Critic 네트웤에서 높은 Q-value를 유도하도록 업데이트
            actor_loss = -critic(states_b, actor(states_b)).mean()
            actor_optimizer.zero_grad()
            actor_loss.backward()
            actor_optimizer.step()
            
            # 타깃 네트웤 soft update: 원본 네트웤 파라미터의 일부를 타깃 네트웤에 반영하여 천천히 업데이트
            """ 오리지널 코드인데 순서를 아래처럼 깔끔하게 변경했다
            for target_param, param in zip(target_actor.parameters(), actor.parameters()):
                target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
            for target_param, param in zip(target_critic.parameters(), critic.parameters()):
                target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
            """
            
            for param, target_param in zip(actor.parameters(), target_actor.parameters()):
                target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
            for param, target_param in zip(critic.parameters(), target_critic.parameters()):
                target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
    
    # 에피소드 진행 중 PRINT_INTERVAL 에피소드마다 결과 출력
    if episode % PRINT_INTERVAL == 0:
        print(f"Episode {episode:3d} | Episode Reward: {episode_reward:.4f}")

print("DDPG 학습 완료")

##############################################
# 8. 백테스팅: 학습된 Actor 네트웤으로 전체 기간에 대해 포트폴리오 수익률 산출
##############################################
def backtest(actor_model, returns_np):
    """
    주어진 학습된 Actor 네트웤을 사용하여 전체 백테스트 기간 동안의 일별 포트폴리오 수익률을 산출.
    - 결정론적 행동 사용: 탐험 노이즈 없이 Actor의 출력(softmax 적용된 투자 비중)을 그대로 사용.
    """
    actor_model.eval()  # 평가 모드로 전환
    env_bt = HistoricalPortfolioEnv(returns_np)  # 백테스트용 환경 초기화
    state = env_bt.reset()
    daily_returns = []
    while True:
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        with torch.no_grad():
            # 결정론적 행동 산출 (탐험 없이)
            action = actor_model(state_tensor).cpu().data.numpy().flatten()
        
        next_state, reward, done, _ = env_bt.step(action) # 환경에서 다음 상태, 보상, 종료 여부 획득
        daily_returns.append(reward)
        if done:
            break
        state = next_state
    return np.array(daily_returns)

# 백테스트: 학습된 DDPG Actor를 사용한 일별 수익률 산출
ddpg_daily_returns = backtest(actor, returns_np)
# 누적 수익률 계산: 매일의 수익률을 누적 곱하여 포트폴리오 성장 곡선 산출
ddpg_cumulative = np.cumprod(1 + ddpg_daily_returns)

##############################################
# 9. 벤치마크 백테스팅: BM (시가총액 비중 포트폴리오)
##############################################
# 벤치마크 포트폴리오의 일별 수익률 계산:
# 각 종목의 수익률에 시가총액 비중을 곱한 후 합산
benchmark_daily = returns_df.dot(benchmark_weights).values[-len(ddpg_daily_returns):]
# 누적 수익률 계산
benchmark_cumulative = np.cumprod(1 + benchmark_daily)

# 백테스트 기간 날짜 (returns_df의 index 중 마지막 부분을 사용)
dates = returns_df.index[-len(ddpg_daily_returns):]

##############################################
# 10. 결과 Plot: BM vs DDPG
##############################################
plt.figure(figsize=(12,6))
# 벤치마크 포트폴리오의 누적 수익률 그래프 (점선 스타일)
plt.plot(dates, benchmark_cumulative, label="Benchmark (BM)", linestyle="--")
# DDPG 에이전트 포트폴리오의 누적 수익률 그래프
plt.plot(dates, ddpg_cumulative, label="DDPG Agent")
plt.xlabel("Dates")
plt.ylabel("Cumulative returns")
plt.title("Backtesting: BM vs DDPG")
plt.legend()
plt.grid(True)
plt.show()